# Obteniendo más información de cada caso positivo por COVID-19 en Perú

En este notebook se busca obtener un dataset de todos los **casos positivos** reportados por COVID-19, y agregar información sobre fallecimiento y vacunación recibida (mediante **"id_persona"**). Es decir se busca unir las 3 bases de datos, de casos positivos, fallecimientos y vacunación recibida en una sola.

Se considera como **vacunado** a todas las personas que recibieron 2 dosis de vacunación.

## A. Lectura y Procesamiento

### Cargar Librerías

In [13]:
import pandas as pd
import numpy as np
from epiweeks import Week
import dask.dataframe as dd
from datetime import datetime

### Diccionarios para codificar variables

In [14]:
dic_sex = {'MASCULINO': 1,'FEMENINO':2}
dic_met = {'PR': 1, 'PCR':2, 'AG':3}
dic_dep = {
    "AMAZONAS"	:1,
    "ANCASH"	:2,
    "APURIMAC"	:3,   
    "AREQUIPA"	:4,
    "AYACUCHO"	:5,   
    "CAJAMARCA"	:6,
    "CALLAO"	:7,
    "CUSCO"	    :8,
    "HUANCAVELICA":9,	
    "HUANUCO"	:10,	
    "ICA"	    :11,	
    "JUNIN"	    :12,
    "LA LIBERTAD":13,
    "LAMBAYEQUE":14,
    "LIMA"	    :15,
    "LORETO"	:16,
    "MADRE DE DIOS":17,
    "MOQUEGUA"	:18,	
    "PASCO"	    :19,
    "PIURA"	    :20,
    "PUNO"	    :21,
    "SAN MARTIN":22,
    "TACNA"	    :23,
    "TUMBES"	:24,
    "UCAYALI"	:25
}

### Dataset de casos positivos

In [15]:
# Leer dataset
url_cas = '../RawData/positivos_covid.csv'
col_cas = ['FECHA_RESULTADO','EDAD', 'DEPARTAMENTO', 'SEXO', 'id_persona', 'METODODX']
df_cas = pd.read_csv(url_cas, sep = ';', usecols = col_cas, dtype={'EDAD': 'float64', 
                                                                   'FECHA_RESULTADO': 'float64'})
# Correguir datos erróneos encontrados en el anáisis exploradorio de "0_ExploratoryDataAnalysis.ipynb"
df_cas = df_cas.dropna()
df_cas = df_cas.astype({'EDAD':'int8', 'FECHA_RESULTADO': 'int32', 'id_persona': 'int32'})

df_cas.drop(df_cas[df_cas.SEXO == '.'].index, inplace = True)
df_cas.drop(df_cas[df_cas.EDAD == -79].index, inplace = True)  # un -9 en edad

# Categorizar edad en grupos etáreos
def edad_cat(df, col):
    """Función para categorizar edad en grupos etáreos"""
    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]
    return np.select(conditions, choices, default=np.nan).astype('int8')
df_cas['edad_cat'] = edad_cat(df_cas, 'EDAD') 

# Reemplazar sexo con un code
df_cas['SEXO'].replace(dic_sex, inplace=True)
df_cas['SEXO'] = df_cas['SEXO'].astype('int8')

# Ordenar columnas
df_cas.rename(columns = {'FECHA_RESULTADO':'fecha_resultado',
                         'EDAD': 'edad',
                         'SEXO': 'sexo',
                         'DEPARTAMENTO': 'departamento',
                         'METODODX': 'metododx'}, inplace=True)
df_cas = df_cas[['id_persona','fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento']] \
         .sort_values(by = ['fecha_resultado']) \
         .reset_index(drop = True)

# Codificar variables
df_cas['departamento'] = df_cas['departamento'].map(dic_dep).astype('int8')  
df_cas['metododx'].replace(dic_met, inplace=True)
df_cas['metododx'] = df_cas['metododx'].astype('int8') 

In [16]:
df_cas

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento
0,34978840,20200306,2,1,25,1,15
1,12211040,20200307,2,1,7,0,15
2,7800643,20200307,2,2,74,6,15
3,12149681,20200307,2,1,25,1,15
4,10629488,20200307,2,1,51,4,15
...,...,...,...,...,...,...,...
2785443,4149932,20220120,3,1,37,2,15
2785444,18341890,20220120,2,1,36,2,15
2785445,26910589,20220120,2,1,31,2,15
2785446,22388651,20220120,2,2,18,1,15


### Dataset de fallecidos

In [17]:
# Leer el dataset
url_fal = '../RawData/fallecidos_covid.csv'
col_fal = ['FECHA_FALLECIMIENTO', 'id_persona']
df_fal = pd.read_csv(url_fal, sep = ';', usecols = col_fal, dtype = {'FECHA_FALLECIMIENTO':'int32', 
                                                                     'EDAD_DECLARADA': 'int8'})
df_fal = df_fal.dropna()  # Drop nan values
df_fal['fallecido'] = 1
df_fal = df_fal.astype({'id_persona': 'int32', 'fallecido': 'int8'})

# Transformar a formato fecha para operar los días
df_fal['fecha'] = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d')

# Agregar la semana epidemiológica y el año
df_fal[['year','epi_week']] = df_fal['fecha'].apply(lambda date : Week.fromdate(date) \
                                                                      .weektuple()) \
                                                                      .tolist()

# Ordenar columnas
df_fal.rename(columns = {'FECHA_FALLECIMIENTO':'fecha_fallecimiento'}, inplace=True)

df_fal

,fecha_fallecimiento,id_persona,fallecido,fecha,year,epi_week
0,20210607,24804325,1,2021-06-07,2021,23
1,20210716,24652285,1,2021-07-16,2021,28
2,20210214,24652408,1,2021-02-14,2021,7
3,20210722,24652744,1,2021-07-22,2021,29
4,20200505,24803428,1,2020-05-05,2020,19
...,...,...,...,...,...,...
200880,20210316,19089970,1,2021-03-16,2021,11
200881,20211213,19156708,1,2021-12-13,2021,50
200882,20210423,19265850,1,2021-04-23,2021,16
200883,20210626,19278368,1,2021-06-26,2021,25


### Buscar en el dataset de casos positivos quienes fallecieron y cuándo

In [18]:
df = df_cas.merge(df_fal, on = 'id_persona', how = 'left')
df['fallecido'].fillna(0, inplace=True)
df['fallecido'] = df['fallecido'].astype('int8')
del df_fal, df_cas

# IMPORTANTE, se repiten pruebas en algunos
df

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fecha_fallecimiento,fallecido,fecha,year,epi_week
0,34978840,20200306,2,1,25,1,15,NaN,0,NaT,NaN,NaN
1,12211040,20200307,2,1,7,0,15,NaN,0,NaT,NaN,NaN
2,7800643,20200307,2,2,74,6,15,NaN,0,NaT,NaN,NaN
3,12149681,20200307,2,1,25,1,15,NaN,0,NaT,NaN,NaN
4,10629488,20200307,2,1,51,4,15,NaN,0,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2785477,4149932,20220120,3,1,37,2,15,NaN,0,NaT,NaN,NaN
2785478,18341890,20220120,2,1,36,2,15,NaN,0,NaT,NaN,NaN
2785479,26910589,20220120,2,1,31,2,15,NaN,0,NaT,NaN,NaN
2785480,22388651,20220120,2,2,18,1,15,NaN,0,NaT,NaN,NaN


In [19]:
def days_between(d1, d2):
    return abs((d2 - d1).days)

# Agregar la diferencia en días desde el primer caso
fecha_min = df['fecha'].min()  
df['time_days'] = df['fecha'].apply(lambda row : days_between(row, fecha_min)+1)
df['time_days'] = df['time_days'].fillna(df.time_days.max()).astype('int')

del df['fecha']
df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento',
         'fallecido', 'time_days', 'fecha_fallecimiento', 'year', 'epi_week']]
df

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,year,epi_week
0,34978840,20200306,2,1,25,1,15,0,678,NaN,NaN,NaN
1,12211040,20200307,2,1,7,0,15,0,678,NaN,NaN,NaN
2,7800643,20200307,2,2,74,6,15,0,678,NaN,NaN,NaN
3,12149681,20200307,2,1,25,1,15,0,678,NaN,NaN,NaN
4,10629488,20200307,2,1,51,4,15,0,678,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2785477,4149932,20220120,3,1,37,2,15,0,678,NaN,NaN,NaN
2785478,18341890,20220120,2,1,36,2,15,0,678,NaN,NaN,NaN
2785479,26910589,20220120,2,1,31,2,15,0,678,NaN,NaN,NaN
2785480,22388651,20220120,2,2,18,1,15,0,678,NaN,NaN,NaN


### Convertir en dask dataframe (ddf) para poder trabajar con el ddf de vacunados

In [20]:
df = dd.from_pandas(df, npartitions=3)

### Dataset de vacunados

In [21]:
# Leer el dataset (dask dataframe ddf)
url_vac = '../RawData/TB_VACUNACION_COVID19.csv'
col_vac = ['id_persona','fecha_vacunacion','dosis', 'id_vacuna']
ddf_vac = dd.read_csv(url_vac, sep = ",", usecols = col_vac, dtype = {'fecha_vacunacion':'int32', 
                                                                      'dosis': 'int8',
                                                                      'id_vacuna': 'int8',
                                                                      'id_persona': 'int64'})

# Convertir la columna 'fecha_vacunacion' a formato fecha y obtener el año y semana epidemiológica
""" ddf_vac = ddf_vac.assign(fecha_vacunacion = dd.to_datetime(ddf_vac['fecha_vacunacion'], 
                                                           format = '%Y%m%d', 
                                                           errors="coerce")) """

ddf_vac = ddf_vac.rename(columns = {'fecha_vacunacion': 'fecha_vacuna'})
ddf_vac = ddf_vac[['id_persona', 'dosis', 'fecha_vacuna', 'id_vacuna']]

ddf_vac.head()

,id_persona,dosis,fecha_vacuna,id_vacuna
0,35101043,2,20211128,5
1,34951963,2,20211129,5
2,34814362,2,20210827,6
3,35213996,2,20211214,6
4,35226113,1,20211208,6


### Separar información de dosis recibidas para juntarlos con nuestro dataframe

In [22]:
df_dosis_1 = ddf_vac[ddf_vac.dosis == 1]
df_dosis_1 = df_dosis_1.rename(columns = {'dosis' : 'dosis_1', 
                                          'fecha_vacuna': 'fecha_vacuna_1', 
                                          'id_vacuna': 'id_vacuna_1'})
df = df.merge(df_dosis_1, on = 'id_persona', how = 'left')
del df_dosis_1

df_dosis_2 = ddf_vac[ddf_vac.dosis == 2]
df_dosis_2['dosis'] = df_dosis_2['dosis'].replace(2, 1)
df_dosis_2 = df_dosis_2.rename(columns = {'dosis' : 'dosis_2', 
                                          'fecha_vacuna': 'fecha_vacuna_2', 
                                          'id_vacuna': 'id_vacuna_2'})
df = df.merge(df_dosis_2, on = 'id_persona', how = 'left')
del df_dosis_2

df_dosis_3 = ddf_vac[ddf_vac.dosis == 3]
df_dosis_3['dosis'] = df_dosis_3['dosis'].replace(3, 1)
df_dosis_3 = df_dosis_3.rename(columns = {'dosis' : 'dosis_3', 
                                          'fecha_vacuna': 'fecha_vacuna_3', 
                                          'id_vacuna': 'id_vacuna_3'})
df = df.merge(df_dosis_3, on = 'id_persona', how = 'left')
del df_dosis_3, ddf_vac

### Convertir nuestro ddf en un pandas dataframe

In [23]:
df = df.compute()

### Rellenar espacios vacíos con 0

In [24]:
# Variable "vacunado" si recibe las 2 dosis
df['vacunado'] = np.where(df['dosis_2'] == 1, 1, 0).astype('int8')

df['dosis_1'] = df['dosis_1'].fillna(0).astype('int8')
df['dosis_2'] = df['dosis_2'].fillna(0).astype('int8')
df['dosis_3'] = df['dosis_3'].fillna(0).astype('int8')

df['id_vacuna_1'] = df['id_vacuna_1'].fillna(0).astype('int8')
df['id_vacuna_2'] = df['id_vacuna_2'].fillna(0).astype('int8')
df['id_vacuna_3'] = df['id_vacuna_3'].fillna(0).astype('int8')

### Ordenar y Guardar el df

In [25]:
df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento', 
         'fallecido', 'time_days', 'fecha_fallecimiento', 'year', 'epi_week',
         'vacunado', 'dosis_1', 'dosis_2', 'dosis_3', 
         'id_vacuna_1', 'id_vacuna_2', 'id_vacuna_3' ,'fecha_vacuna_1','fecha_vacuna_2', 'fecha_vacuna_3']]

### Eliminar duplicados

In [26]:
# Ordenar de acuerdo al id_de persona y fecha de resultado
df = df.sort_values( by = ['id_persona', 'fecha_resultado'])

In [27]:
# Vemos todos los duplicados, ya que debería haber un solo id para cada persona
df[df.id_persona.duplicated() == True]

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,...,vacunado,dosis_1,dosis_2,dosis_3,id_vacuna_1,id_vacuna_2,id_vacuna_3,fecha_vacuna_1,fecha_vacuna_2,fecha_vacuna_3
43100,227,20210421,3,1,81,7,25,0,678,NaN,...,1,1,1,0,6,6,0,20211202.0,20211223.0,NaN
47951,324,20210525,3,2,54,4,25,0,678,NaN,...,1,1,1,1,2,1,5,20210824.0,20210927.0,20220106.0
44014,599,20210424,2,1,58,4,25,0,678,NaN,...,1,1,1,1,5,5,5,20210526.0,20210622.0,20220114.0
63316,839,20220114,2,2,59,4,25,0,678,NaN,...,1,1,1,1,2,2,5,20210708.0,20210804.0,20211222.0
41779,1665,20210413,2,2,60,5,25,1,407,20210424.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20190,37275477,20200925,2,1,28,1,15,0,678,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
36841,37281588,20210321,2,1,19,1,15,0,678,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
61145,37285950,20220112,2,1,26,1,20,1,54,20200506.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
61710,37286078,20220112,2,2,22,1,15,1,79,20200531.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [28]:
# Tomamos uno de ejemplo
df[df.id_persona == 227]

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,...,vacunado,dosis_1,dosis_2,dosis_3,id_vacuna_1,id_vacuna_2,id_vacuna_3,fecha_vacuna_1,fecha_vacuna_2,fecha_vacuna_3
43063,227,20210421,3,1,81,7,25,0,678,NaN,...,1,1,1,0,6,6,0,20211202.0,20211223.0,NaN
43100,227,20210421,3,1,81,7,25,0,678,NaN,...,1,1,1,0,6,6,0,20211202.0,20211223.0,NaN


Podremos conservar el último duplicado, ya es es la fecha más reciente de un caso positivo. Estos duplicados representan varios contagios de la misma persona, sin embargo se toman como un nuevo contagio.

En este caso para el propósito del estudio nos interesa la mortalidad y la vacunación, por ello no se toma en cuenta el número de veces de contagio o fechas de la misma, si no todos los datos e id del paciente.

In [29]:
# Eliminamos los duplicados
df = df.drop_duplicates('id_persona', keep='last').reset_index(drop = True)

In [30]:
df

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,...,vacunado,dosis_1,dosis_2,dosis_3,id_vacuna_1,id_vacuna_2,id_vacuna_3,fecha_vacuna_1,fecha_vacuna_2,fecha_vacuna_3
0,7,20200720,1,1,54,4,16,0,678,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,11,20200701,2,2,56,4,25,0,678,NaN,...,1,1,1,1,6,6,5,20210218.0,20210317.0,20211028.0
2,12,20210413,3,2,62,5,25,1,403,20210420.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
3,30,20210628,2,2,55,4,25,0,678,NaN,...,1,1,1,0,5,5,0,20211014.0,20211124.0,NaN
4,37,20210429,3,1,74,6,25,1,421,20210508.0,...,0,1,0,0,1,0,0,20210426.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2643702,37293491,20201004,2,1,41,3,15,0,678,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2643703,37293520,20200516,1,2,34,2,15,0,678,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2643704,37293584,20200513,2,1,80,7,15,1,75,20200527.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2643705,37293776,20210114,2,2,81,7,14,1,303,20210110.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2643707 entries, 0 to 2643706
Data columns (total 22 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id_persona           int32  
 1   fecha_resultado      int32  
 2   metododx             int8   
 3   sexo                 int8   
 4   edad                 int8   
 5   edad_cat             int8   
 6   departamento         int8   
 7   fallecido            int8   
 8   time_days            int64  
 9   fecha_fallecimiento  float64
 10  year                 float64
 11  epi_week             float64
 12  vacunado             int8   
 13  dosis_1              int8   
 14  dosis_2              int8   
 15  dosis_3              int8   
 16  id_vacuna_1          int8   
 17  id_vacuna_2          int8   
 18  id_vacuna_3          int8   
 19  fecha_vacuna_1       float64
 20  fecha_vacuna_2       float64
 21  fecha_vacuna_3       float64
dtypes: float64(6), int32(2), int64(1), int8(13)
memory usage: 194.1 MB

In [32]:
df.to_csv('../Data/DP3_covid19-peru_casos-positivos_+_info-fallecimiento_+_info-vacunas.csv',  index=False)